<a href="https://colab.research.google.com/github/msmekka/ncssm-summer25-cyber/blob/main/Secure_AI_Spam_Classifier_Colab_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Secure Your Data, Power Your AI
Welcome to your project notebook! You will build a spam classifier using anonymized data. Work through each section and complete the code where prompted.

## 🔍 Step 1: Load Your Dataset
We're using the SMS Spam Collection dataset from Kaggle. Upload and load it below.

In [ ]:
# Import libraries
import pandas as pd

# Load dataset (replace with actual file path or use Kaggle API)
# df = pd.read_csv('/path/to/dataset.csv')
#df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv', sep='\t', header=None, names=['label', 'message'])
df = pd.read_csv('/content/sample_data/spam.csv', encoding='latin1')
print(df.shape)
df.head()

(5572, 5)


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## 🛡️ Step 2: Explore for Sensitive Data
Check for names, numbers, emails, or other personally identifiable information (PII).

In [ ]:
# Use regex to identify PII patterns
import re

def find_phone_numbers(text):
    return re.findall(r'\d{11}', text)

df['phone_numbers'] = df['v2'].apply(find_phone_numbers)
df['phone_numbers'].head()

,phone_numbers
0,[]
1,[]
2,[08452810075]
3,[]
4,[]


## 🔒 Step 3: Anonymize Sensitive Data
Use masking or redaction to protect the data.

In [ ]:
# Example: replace names with [NAME] (use your own logic to find names)

def anonymize_message(msg):
    #msg = re.sub(r'\b\d{11}\b', '[PHONE]', msg)
    msg = re.sub(r'\d{11}', '[PHONE]', msg)
    # Add other anonymization logic here
    return msg

df['message_anonymized'] = df['v2'].apply(anonymize_message)
df[['v2', 'message_anonymized']].head()

,v2,message_anonymized
0,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ..."
1,Ok lar... Joking wif u oni...,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro..."


## 🤖 Step 4: Train a Spam Classifier
Use Scikit-learn to train a simple model on the anonymized messages.

In [ ]:
# Preprocess and vectorize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    df['message_anonymized'], df['v1'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       965
        spam       0.98      0.76      0.86       150

    accuracy                           0.97      1115
   macro avg       0.97      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115



## 🧾 Step 5: Reflect & Document
Answer the following:
- What types of PII did you find and remove?
- How did you anonymize them?
- Did anonymization affect model accuracy?
- What would the risks be if the data were left unprotected?